In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jul  5 12:42:38 2018

@author: grace
"""
#reference website
#https://www.wildcardconsulting.dk/useful-information/teaching-computers-molecular-creativity/
#Text classification using scikit-learn, python and NLTK

#https://towardsdatascience.com/machine-learninag-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

#Test dataset: 20 newsgroup
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
twenty_train.target_names

#extract features from text files-bag of words
#segment each text files into words and count of times each word occurs in each documents and assign integer id to each word
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape #(number of sampels, number of features)

#counting the number of words give more weight to longer documents
#To avid this, TF-term frequence = count(word)/total words in each document is applied
#To reduce the weight of more common words, TF-IDF(term freqquency times inverse document frequency) is used
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

#Naive Bayse
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

#Pipeline
from sklearn.pipeline import Pipeline
text_classifier = Pipeline([('vector', CountVectorizer(stop_words='english')),
                            ('tfidf', TfidfTransformer()),
                            ('classifier', MultinomialNB())])
text_classifier.fit(twenty_train.data, twenty_train.target)

import numpy as np

twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_classifier.predict(twenty_test.data)
np.mean(predicted==twenty_test.target)

0.8169144981412639

In [38]:
#stemming: process of reducing inflated(derived) words to their word stem
import nltk
# nltk.download()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english', ignore_stopwords=True)

# class StemmedCountVectorizer(CountVectorizer):
#     def build_analylzer(self):
#         analyzer = super(StemmedCountVectorizer, self).build_analyzer()
#         return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
# stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

analyzer = CountVectorizer(stop_words='english').build_analyzer()
stemmed_count_vect = lambda doc :[stemmer.stem(w) for w in analyzer(doc)]
stemmed_count_vect.fit()

AttributeError: 'function' object has no attribute 'fit'

In [29]:
# [stemmer.stem(w) for w in analyzer("watched movies, drunk water and move mouse, my, me \n they")]

# stemmed_count_vect("watched movies, drunk water and move mouse, my, me \n they")
# stemmer.stem('broke')
stemmed_count_vect("broken, broke, break")

['broken', 'broke', 'break']

In [36]:
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect), ('tfidf', TfidfTransformer()),('mnb',MultinomialNB(fit_prior=False))])

In [20]:
text_mnb_stemmed = text_mnb_stemmed.fit(twenty_train.data, twenty_train.target)

In [22]:
predicted_mnb_stemmed = text_mnb_stemmed.predict(twenty_test.data)

In [23]:
np.mean(predicted_mnb_stemmed==twenty_test.target)

0.8214285714285714